In [1]:
%load_ext autoreload
%autoreload 2

import pdfplumber
import re

from utils import create_new_sheet, write_content, format_sheet

import os
os.remove('../google_token.json')

In [2]:
############ Update the following parameters ##################
FILEPATH = "../data/May statement.pdf"
NEW_SHEET_NAME = "May 2025"

In [3]:
SPREADSHEET_ID = "1Ik2lyOO5MN9a8-hEUf4sRqUlP3WcN36ItekQSGV4NxY"

In [4]:
pdf_lines = []
with pdfplumber.open(FILEPATH) as pdf:
    for page_id in range(len(pdf.pages)):
        page = pdf.pages[page_id]
        pdf_lines.extend(page.extract_text_lines())
pdf_lines = [line['text'] for line in pdf_lines]

In [5]:
import pickle
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

# lr_model = Pipeline([('vect', CountVectorizer()),
#                ('tfidf', TfidfTransformer()),
#                ('clf', LogisticRegression(n_jobs=1, C=1e5)),
#               ])

lr_model = pickle.load(open('pickle/lr_model.pkl', 'rb'))

In [6]:
id_to_label = {0: 'Regular', 1: 'Trip', 2: 'Hobby'}

In [7]:
regex_pattern = "^(\d\d/\d\d.*) (-*[\d\.,]+)$"
accumulated_content = []
for line in pdf_lines:
    if "AUTOMATIC PAYMENT" in line:
        continue
    re_search = re.search(regex_pattern, line)
    if re_search is not None:
        transaction_name = re_search.group(1)
        expense_type_id = lr_model.predict([transaction_name])[0]
        expense_type = id_to_label[expense_type_id] if expense_type_id in id_to_label else 'Regular'
        cost = float(re_search.group(2).replace(',', ''))
        accumulated_content.append([transaction_name, cost, expense_type])

In [8]:
# Sort the content according to the transaction date
accumulated_content = sorted(accumulated_content, key=lambda x: x[0].split(' ')[0])

for i in range(len(accumulated_content)):
    row_id = i + 2
    regular_cost = f'=IF(C{row_id}="Regular", B{row_id}, 0)'
    trip_cost = f'=IF(C{row_id}="Trip", B{row_id}, 0)'
    hobby_cost = f'=IF(C{row_id}="Hobby", B{row_id}, 0)'
    accumulated_content[i].append(regular_cost)
    accumulated_content[i].append(trip_cost)
    accumulated_content[i].append(hobby_cost)

accumulated_content = [['Transaction', 'Cost', 'Type', 'Regular expenses', 'Trip expenses', 'Hobby expenses']] + accumulated_content + [['', f'=SUM(B2:B{row_id})', '', f'=SUM(D2:D{row_id})', f'=SUM(E2:E{row_id})', f'=SUM(F2:F{row_id})', f'=SUM(D{row_id+1}:F{row_id+1})']]

In [9]:
# Sort the content according to the transaction date
# accumulated_content = [accumulated_content[0]] + sorted(accumulated_content[1:-1], key=lambda x: x[0].split(' ')[0]) + [accumulated_content[-1]]

In [10]:
s = ""
for row in accumulated_content:
    print('***'.join([str(item) for item in row]))


Transaction***Cost***Type***Regular expenses***Trip expenses***Hobby expenses
05/08 SAFEWAY #1990 SANTA CLARA CA***33.3***Regular***=IF(C2="Regular", B2, 0)***=IF(C2="Trip", B2, 0)***=IF(C2="Hobby", B2, 0)
05/09 MCDONALD'S F17109 REDWOOD CITY CA***11.19***Regular***=IF(C3="Regular", B3, 0)***=IF(C3="Trip", B3, 0)***=IF(C3="Hobby", B3, 0)
05/09 SP RACQUETSTORE.COM 140-82871099 CA***425.46***Regular***=IF(C4="Regular", B4, 0)***=IF(C4="Trip", B4, 0)***=IF(C4="Hobby", B4, 0)
05/10 DISH N DASH SAN JOSE DISHDASH.OLO. CA***75.27***Regular***=IF(C5="Regular", B5, 0)***=IF(C5="Trip", B5, 0)***=IF(C5="Hobby", B5, 0)
05/10 SQ *FIVE SPICE INDIAN GRO San Jose CA***132.91***Regular***=IF(C6="Regular", B6, 0)***=IF(C6="Trip", B6, 0)***=IF(C6="Hobby", B6, 0)
05/10 SAFEWAY #1990 SANTA CLARA CA***62.98***Regular***=IF(C7="Regular", B7, 0)***=IF(C7="Trip", B7, 0)***=IF(C7="Hobby", B7, 0)
05/10 THE HOME DEPOT 630 SANTA CLARA CA***14.4***Regular***=IF(C8="Regular", B8, 0)***=IF(C8="Trip", B8, 0)***=IF(C8=

In [11]:
s

''

In [12]:
# Export to the spreadsheet.
create_new_sheet_response = create_new_sheet(SPREADSHEET_ID, NEW_SHEET_NAME)
sheet_id = create_new_sheet_response['replies'][0]['addSheet']['properties']['sheetId']
write_response = write_content(SPREADSHEET_ID, NEW_SHEET_NAME, accumulated_content)
format_sheet_response = format_sheet(SPREADSHEET_ID, sheet_id)

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=694301136407-7lfplojamr3flfdqoo629e5mf15pv8s7.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A51783%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fspreadsheets&state=LQGVXEDLvpkbuiciF5Qd9dsMxeKAF1&access_type=offline
Created a new sheet result: {'spreadsheetId': '1Ik2lyOO5MN9a8-hEUf4sRqUlP3WcN36ItekQSGV4NxY', 'replies': [{'addSheet': {'properties': {'sheetId': 457293470, 'title': 'May 2025', 'index': 23, 'sheetType': 'GRID', 'gridProperties': {'rowCount': 1000, 'columnCount': 26}}}}]}
Exported content to 391 cells.
Format sheet result: {'spreadsheetId': '1Ik2lyOO5MN9a8-hEUf4sRqUlP3WcN36ItekQSGV4NxY', 'replies': [{}, {}, {}]}
